In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import beamsim

# Set seaborn style for beautiful plots
sns.set_style("whitegrid")
sns.set_palette("husl")
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["font.size"] = 11

In [ ]:
def plot1(items, ax: plt.Axes = None, topology_name=""):
    _, _, validator_count, snark1_threshold, snark2_threshold = beamsim.filter_report(
        items, "info"
    )[0]
    snark1_sent = beamsim.get_snark1_sent(items)
    snark1_received = beamsim.get_snark1_received(items)
    snark2_sent = beamsim.filter_report(items, "snark2_sent")

    show = ax is None
    if ax is None:
        _, ax = plt.subplots(figsize=(10, 6))

    # Use seaborn color palette
    colors = sns.color_palette("husl", 3)

    # Plot with enhanced styling
    ax.step(
        *snark1_sent,
        where="post",
        color=colors[0],
        linewidth=2.5,
        label="SNARK1 Sent (Cumulative)",
        alpha=0.8,
    )
    ax.step(
        [0, *beamsim.time_axis(snark1_received)],
        [0, *[x[2] for x in snark1_received]],
        where="post",
        color=colors[1],
        linewidth=2.5,
        label="SNARK1 Received",
        alpha=0.8,
    )

    # Enhanced reference lines
    ax.axhline(
        validator_count,
        linestyle="--",
        color=colors[0],
        alpha=0.7,
        linewidth=2,
        label=f"Validator Count ({validator_count})",
    )
    ax.axhline(
        snark1_threshold,
        linestyle="--",
        color=colors[0],
        alpha=0.7,
        linewidth=2,
        label=f"SNARK1 Threshold ({snark1_threshold})",
    )
    ax.axhline(
        snark2_threshold,
        linestyle="--",
        color=colors[1],
        alpha=0.7,
        linewidth=2,
        label=f"SNARK2 Threshold ({snark2_threshold})",
    )

    if snark2_sent:
        snark2_time = snark2_sent[0][0]
        ax.axvline(
            snark2_time,
            linestyle=":",
            color=colors[2],
            alpha=0.8,
            linewidth=2,
            label=f"SNARK2 Sent Time ({snark2_time})",
        )

    # Enhanced styling
    ax.set_xlabel("Time", fontweight="bold")
    ax.set_ylabel("Count", fontweight="bold")
    if topology_name:
        ax.set_title(
            f"SNARK Distribution - {topology_name.title()} Topology",
            fontweight="bold",
            fontsize=14,
        )

    ax.legend(frameon=True, fancybox=True, shadow=True)
    ax.grid(True, alpha=0.3)

    if show:
        plt.tight_layout()
        plt.show()


def plot1_topologies(topologies=beamsim.topologies, **run_kwargs):
    # Create enhanced subplots with better spacing
    fig, axes = plt.subplots(nrows=len(topologies), figsize=(12, 15), sharex=True)
    fig.suptitle(
        "SNARK Distribution Across Different Network Topologies",
        fontsize=16,
        fontweight="bold",
        y=0.95,
    )

    for topology, ax in zip(topologies, axes):
        plot1(
            beamsim.run(**run_kwargs, t=topology), ax, beamsim.topology_name[topology]
        )

    plt.tight_layout()
    plt.subplots_adjust(top=0.92)
    plt.show()

In [ ]:
def plot2(items, ax: plt.Axes = None, topology_name=""):
    metrics = beamsim.Metrics(items)

    show = ax is None
    if ax is None:
        _, ax = plt.subplots(figsize=(10, 6))

    # Create a more structured data approach
    time_points = range(metrics.t)
    colors = sns.color_palette("Set2", 3)

    for y, label, color in zip(metrics.bytes_sent_role_avg, beamsim.role_name, colors):
        ax.plot(
            time_points,
            y,
            label=label,
            linewidth=3,
            color=color,
            marker="o",
            markersize=4,
            alpha=0.8,
        )

    ax.set_xlabel("Time Step", fontweight="bold")
    ax.set_ylabel("Average Bytes Sent", fontweight="bold")
    if topology_name:
        ax.set_title(
            f"Network Traffic by Role - {topology_name} Topology",
            fontweight="bold",
            fontsize=14,
        )

    ax.legend(frameon=True, fancybox=True, shadow=True, loc="upper left")
    ax.grid(True, alpha=0.3)

    # Add some styling enhancements
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    if show:
        plt.tight_layout()
        plt.show()


def plot2_topologies(topologies=beamsim.topologies, **run_kwargs):
    # Create a comprehensive comparison plot
    fig, axes = plt.subplots(
        nrows=len(topologies), figsize=(14, 12), sharex=True, sharey=True
    )
    fig.suptitle(
        "Network Traffic Analysis Across Different Topologies",
        fontsize=16,
        fontweight="bold",
        y=0.95,
    )

    # Run simulations and plot
    for topology, ax in zip(topologies, axes):
        items = beamsim.run(**run_kwargs, t=topology)
        plot2(items, ax, beamsim.topology_name[topology])

    plt.tight_layout()
    plt.subplots_adjust(top=0.92)
    plt.show()

In [ ]:
def plot3_topologies(topologies=beamsim.topologies, **run_kwargs):
    # Additional: Create separate plots for sent and received bytes
    _, axes = plt.subplots(2, 1, figsize=(12, 16))

    def plot(ax: plt.Axes, f_data_role, title, ylabel):
        # Collect data for all topologies
        all_data = []
        for topology in topologies:
            items = beamsim.run(**run_kwargs, t=topology)
            metrics = beamsim.Metrics(items)

            # Calculate total bytes sent and received for each role
            for data, role in zip(f_data_role(metrics), beamsim.role_name):
                all_data.append(
                    {
                        "Topology": beamsim.topology_name[topology],
                        "Role": role,
                        "Total Bytes (MB)": data[-1] / (1024 * 1024),
                    }
                )

        # Create DataFrames
        df = pd.DataFrame(all_data)

        sns.barplot(data=df, x="Topology", y="Total Bytes (MB)", hue="Role", ax=ax)
        ax.set_title(title, fontweight="bold", fontsize=14)
        ax.set_ylabel(ylabel, fontweight="bold")
        ax.set_xlabel("Network Topology", fontweight="bold")
        ax.legend(title="Node Role", frameon=True, fancybox=True, shadow=True)

    # Plot sent bytes
    plot(
        axes[0],
        lambda metrics: metrics.bytes_sent_role,
        "Total Sent Network Traffic by Topology and Role",
        "Total Bytes Sent (MB)",
    )
    # Plot received bytes
    plot(
        axes[1],
        lambda metrics: metrics.bytes_received_role,
        "Total Received Network Traffic by Topology and Role",
        "Total Bytes Received (MB)",
    )

    plt.tight_layout()
    plt.show()

In [ ]:
def plot4_topologies(topologies=beamsim.topologies, **run_kwargs):
    import warnings

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        all_data = []
        for topology in topologies:
            items = beamsim.run(**run_kwargs, t=topology)
            metrics = beamsim.Metrics(items)
            for data_role, direction in [
                (metrics.bytes_sent_role_avg, "Outgoing"),
                (metrics.bytes_received_role_avg, "Incoming"),
            ]:
                for data, role in zip(data_role, beamsim.role_name):
                    window = 1000  # 1000ms window
                    windows = len(data) - window
                    if windows > 0:
                        peak = max(data[i + window] - data[i] for i in range(windows))
                    else:
                        peak = data[-1]
                    peak_mbps = (peak * 8) / (1000 * 1000)  # bytes -> bits -> Mbps
                    all_data.append(
                        {
                            "Topology": beamsim.topology_name[topology],
                            "Role": role,
                            "Peak": peak_mbps,
                            "Direction": direction,
                        }
                    )
        df = pd.DataFrame(all_data)
        g = sns.catplot(
            data=df,
            x="Topology",
            y="Peak",
            hue="Role",
            col="Direction",
            kind="bar",
            ci=None,
            height=7,
            aspect=1.1,
            dodge=True,
            legend_out=False,
        )
        g.set_titles("{col_name} Traffic")
        g.set_axis_labels("Network Topology", "Peak Traffic (Mbps)")
        g.figure.suptitle(
            "Peak Network Traffic by Topology, Role, and Direction",
            fontweight="bold",
            fontsize=15,
            y=1.05,
        )
        for ax in g.axes.flat:
            ax.set_ylabel("Peak Traffic (Mbps)", fontweight="bold")
            ax.set_xlabel("Network Topology", fontweight="bold")
            ax.grid(True, axis="y", alpha=0.3)
        g.add_legend(
            title="Role", frameon=True, fancybox=True, shadow=True
        )  # Ensure legend is displayed
        plt.tight_layout()
        plt.show()

In [ ]:
# Default YAML Configuration for Simulations
yaml_config = """
backend: ns3-direct
topology: gossip
shuffle: false

random_seed: 42

roles:
  group_count: 10
  group_validator_count: 64
  global_aggregator_count: 1
  group_local_aggregator_count: 10

gossip:
  mesh_n: 6
  non_mesh_n: 4

consts:
  signature_time: 20ms
  signature_size: 3072
  snark_size: 131072
  snark1_threshold: 0.9
  snark2_threshold: 0.66
  aggregation_rate_per_sec: 1000
  snark_recursion_aggregation_rate_per_sec: 100
  pq_signature_verification_time: 3ms
  snark_proof_verification_time: 10ms

network:
  direct:
    bitrate:
      min: 10000000
      max: 100000000
    delay:
      min: 10ms
      max: 100ms
"""

# Write the YAML config to a temporary file for use with simulations
# Cache logic to avoid creating duplicate files for the same config
import tempfile
import os
import hashlib

# Generate a hash of the YAML config to check if we already have a file for it
yaml_hash = hashlib.md5(yaml_config.encode()).hexdigest()

# Check if we already have a cached file for this configuration
if "yaml_config_cache" not in globals():
    yaml_config_cache = {}

if yaml_hash in yaml_config_cache and os.path.exists(yaml_config_cache[yaml_hash]):
    # Reuse existing file
    yaml_config_path = yaml_config_cache[yaml_hash]
else:
    # Create a new temporary file with the YAML config
    with tempfile.NamedTemporaryFile(mode="w", suffix=".yaml", delete=False) as f:
        f.write(yaml_config)
        yaml_config_path = f.name

    # Cache the file path for this configuration
    yaml_config_cache[yaml_hash] = yaml_config_path

In [ ]:
# Run simulations using YAML config as defaults
# CLI flags can override the YAML configuration values
# The simulator binary loads YAML config first, then applies CLI flag overrides

# Optional CLI flag overrides (uncomment and modify as needed)
run_kwargs = dict(
    c=yaml_config_path,
    # Uncomment any of these to override the YAML config defaults:
    # b="ns3-direct",    # backend override
    # g=10,              # groups override
    # gv=128,            # group validators override
    mpi=True,  # enable MPI: set to False to disable, alternatively set to number of processes
    # la=1,              # local aggregators override
    # ga=1,              # global aggregators override
    # shuffle=False,     # shuffle override
)

topologies = ["direct", "gossip", "grid"]

plot1_topologies(topologies, **run_kwargs)
plot2_topologies(topologies, **run_kwargs)
plot3_topologies(topologies, **run_kwargs)
plot4_topologies(topologies, **run_kwargs)